This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [3]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Lattice axioms

In [4]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

## Bounded lattice axioms

In [9]:
bot = read_expr('Or(x, Bot) = x')
top = read_expr('And(x, Top) = x')

bounded_lattice = lattice + [bot, top]

## Distributive lattice axioms

In [18]:
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

## Exclusion axioms

In [21]:
excl_cumm = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(And(x1, x2), And(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

event_space = dist_bounded_lattice + [excl_cumm, excl_symm]

## Possibility axioms

In [39]:
#define conflict:
confl_def = read_expr('confl(x, y) <-> exists z1.(And(z1, x) = x & exists z2.(And(z2, y) = y & Excl(z1, z2)))')

#define possibilities:
int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(And(y, z) = z & int_poss(z) & -(confl(z,x))))')

#define world:
world_def = read_expr('world(x) <-> (int_poss(x) & all y.((And(x, y) = y) -> -int_poss(y)))')

#Harmony
poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

#Nirvana
poss_nirv = read_expr('all x.(-confl(Bot, x))')

#Rashōmon
poss_rshm = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(And(x,y)))')

#Cosmopolitanism
poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & And(x, y) = y))')

event_space = dist_bounded_lattice + [excl_cumm, excl_symm, confl_def, int_poss_def, ext_poss_def, world_def, poss_harm, poss_nirv, poss_rshm, poss_cosm]

## Theorems

### Plenitude

In [40]:
#check plenitude
goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 5735 was started by guzzler on oahzzdepbm,
Tue Jun  7 19:59:19 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 32.


6 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
10 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
11 (all x (int_poss(x) -> ext_poss(x))).  [goal].
13 -world(x) | int_poss(x).  [clausify(6)].
14 -world(x) | And(x,y) != y | -int_poss(y).  [clausify(6)].
16 -int_poss(x) | world(f6(x)).  [claus

### Manicaeism

In [41]:
#check Manichaeism
goal = read_expr('all x.((world(x) -> all y.(And(x, y) = x | confl(x, y))))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 5737 was started by guzzler on oahzzdepbm,
Tue Jun  7 19:59:22 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 13.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 36.


6 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
10 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
11 (all x (world(x) -> (all y (And(x,y) = x | confl(x,y))))).  [goal].
19 -world(x) | int_poss(x).  [clausify(6)].
20 -world(x) | And(x,y) != y | -int_poss(y).  [clausify(6)].
22 -int_poss(x) | 